In [5]:
import pandas as pd 
import numpy as np
import sklearn as sklearn
import os as os

import matplotlib.pyplot as plt
import seaborn as sns
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import re as re

if os.getlogin()=="JVARGH7":
    path_equity_precision_llm_folder = "C:/Cloud/OneDrive - Emory University/Papers/Global Equity in Diabetes Precision Medicine LLM"
    path_equity_precision_llm_repo =  'C:/code/external/equity_precision_llm'

elif os.getlogin()=='aamnasoniwala':
    path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
    path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'

path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'

excel_path = path_equity_precision_llm_folder + "/llm training/Unclassified Splits/Unattributable Part "
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

execfile(path_equity_precision_llm_repo + "/functions/crosstab_summary.py")

In [6]:
excel_path_unattributable = path_equity_precision_llm_folder + "//llm training/epldat03_Unattributable Data.csv"
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

execfile(path_equity_precision_llm_repo + "/functions/clean_input.py")
execfile(path_equity_precision_llm_repo + "/functions/crosstab_summary.py")
execfile(path_equity_precision_llm_repo + "/functions/standardize_population.py")
execfile(path_equity_precision_llm_repo + "/functions/adjusted_source_population_match.py")

In [7]:
input_unattributable = pd.read_csv(excel_path_unattributable)
input_unattributable.head()

,PMID,MeSH,Abstract,Title,year
0,18008027,"Albuminuria, Angiotensin II Type 1 Receptor Bl...",The purpose of the present study was to invest...,Renoprotective effects of telmisartan on renal...,2013
1,18205795,"Antigens, Tumor-Associated, Carbohydrate, Bloo...",CA 72-4 is one of the blood group carbohydrate...,CA 72-4 levels in patients with type 2 diabete...,2011
2,18378359,"Age Distribution, Age Factors, Aged, 80 and ov...",The aim of this study was to evaluate the asso...,Vascular risk factors and dementia in the gene...,2010
3,18626579,"Bioreactors, Biotechnology, Fermentation, Gluc...",The production of propionic acid by Propioniba...,Kinetic analysis and pH-shift control strategy...,2010
4,18651247,"Bioreactors, Butyric Acid, Cells, Immobilized,...",Butyric acid has many applications in chemical...,Production of butyric acid from glucose and xy...,2010


In [8]:
import pandas as pd
import re

n_json_splits = 11
# Concatenate multiple CSVs
results = None
for scenario in range(1, n_json_splits + 1):
    file_path = f"{path_equity_precision_llm_folder}/llm training/Unclassified Splits/Unattributable Part {scenario}_results.csv"
    temp_df = pd.read_csv(file_path)
    temp_df = temp_df[['pmid', 'title', 'gpt_precision_medicine', 'gpt_diabetes', 'gpt_primary_study', 'gpt_source_population']]

    if results is None:
        results = temp_df
    else:
        results = pd.concat([results, temp_df], ignore_index=True)

# Merge datasets
merged_df_unattributable = input_unattributable.merge(results, left_on='PMID', right_on='pmid', how='left')
merged_df_unattributable['gpt_source_population'] = merged_df_unattributable['gpt_source_population'].apply(standardize_population)

In [9]:
unique_source_populations = merged_df_unattributable['gpt_source_population'].unique()
print(unique_source_populations)

['Unknown' 'we' 'ea' 'na' 'mena' 'cee'
 'ea, ssa, sa, we, na, cee, mena, seap, ca, lac' 'sa' 'ssa' 'na, lac'
 'lac' 'seap' 'ea, sa' 'ea, we' 'excluded' 'we, ssa'
 'not applicable (non-human study)' 'sa, ssa, we' 'ssa, we' 'na, we'
 'lac, ssa' 'ns' 'sa, we' 'lac, mena, we' 'sa, na' 'ssa, we, ea, lac'
 'ea, na' 'ea, sa, we'
 'not applicable (study uses 3t3-l1 preadipocytes, which are a mouse cell line)'
 'mena, sa' 'we, na' 'na, we, seap' 'we, sa' 'u' 'we, ea, seap, lac'
 'ea, seap' 'ssa, na' 'cee, we' 'undetermined' 'lac, mena'
 'not applicable (study uses non-human animal models)' 'ca'
 'ca, mena, ea, we' 'na, ea, ssa, lac' 'not applicable (animal model)'
 'na, seap, lac, ssa, we' 'na, cee, we' 'unspecified'
 'not applicable (animal model study)'
 'not applicable (study based on cell models, not humans)' 'na, ssa'
 'ea, ssa' 'not applicable (study is on cell lines, not humans)' 'we, ea'
 'seap, we' 'we, mena' 'we, ssa, lac, mena' 'we, na, lac'
 'na, lac, ssa, we' 'mena, we' 'we, sa, ss

In [10]:
# Check the output below
merged_df_unattributable.head()

,PMID,MeSH,Abstract,Title,year,pmid,title,gpt_precision_medicine,gpt_diabetes,gpt_primary_study,gpt_source_population
0,18008027,"Albuminuria, Angiotensin II Type 1 Receptor Bl...",The purpose of the present study was to invest...,Renoprotective effects of telmisartan on renal...,2013,18008027.0,Renoprotective effects of telmisartan on rena...,no,yes,yes,Unknown
1,18205795,"Antigens, Tumor-Associated, Carbohydrate, Bloo...",CA 72-4 is one of the blood group carbohydrate...,CA 72-4 levels in patients with type 2 diabete...,2011,18205795.0,CA 72-4 levels in patients with type 2 diabet...,no,yes,yes,Unknown
2,18378359,"Age Distribution, Age Factors, Aged, 80 and ov...",The aim of this study was to evaluate the asso...,Vascular risk factors and dementia in the gene...,2010,18378359.0,Vascular risk factors and dementia in the gen...,no,yes,yes,we
3,18626579,"Bioreactors, Biotechnology, Fermentation, Gluc...",The production of propionic acid by Propioniba...,Kinetic analysis and pH-shift control strategy...,2010,18626579.0,Kinetic analysis and pH-shift control strateg...,no,no,yes,Unknown
4,18651247,"Bioreactors, Butyric Acid, Cells, Immobilized,...",Butyric acid has many applications in chemical...,Production of butyric acid from glucose and xy...,2010,18651247.0,Production of butyric acid from glucose and x...,no,no,yes,Unknown


In [12]:
import pandas as pd

# Define a function to categorize exclusion reasons
def excluded(row):
    if pd.isna(row['Title']) or pd.isna(row['Abstract']) or pd.isna(row['MeSH']):
        return "Empty"

# Apply the exclusion categorization
merged_df_unattributable['exclusion_reason'] = merged_df_unattributable.apply(excluded, axis=1)

# Create a crosstab summary function
def crosstab_summary(merged_df_unattributable, categories, region_col='gpt_source_population'):
    """
    Generate a crosstab summary of studies categorized by multiple conditions, stratified by region.
    
    :param df: DataFrame containing the study data
    :param categories: List of column names representing different study categories
    :param region_col: Column name representing the region for stratification
    :return: DataFrame summarizing counts of different study combinations by region
    """
    merged_df_unattributable['combined_category'] = merged_df_unattributable[categories].astype(str).agg('_'.join, axis=1)
    summary = pd.crosstab(merged_df_unattributable['combined_category'], merged_df_unattributable[region_col])
    summary.reset_index(inplace=True)
    return summary

# Define categories
categories = ['gpt_precision_medicine', 'gpt_diabetes', 'gpt_primary_study']

# Generate crosstab summary stratified by region
summary_merged_df_unattributable = crosstab_summary(merged_df_unattributable, categories, region_col='gpt_source_population')

# Save the output
output_path = path_equity_precision_llm_repo + '/preprocessing/epl05_combined_output_Unattributable.csv'
summary_merged_df_unattributable.to_csv(output_path, index=False)

# Display summary
display(summary_merged_df_unattributable)

gpt_source_population,combined_category,Unknown,[ca/cee/ea/lac/mena/seap/sa/ssa/na/we],ca,"ca, cee, ea, lac, mena, seap, sa, ssa","ca, cee, ea, lac, mena, seap, sa, ssa, na, we","ca, cee, ea, lac, mena, seap, sa, ssa, we, na","ca, cee, ea, lac, na, sa, ssa, we","ca, cee, ea, mena, ssa, we","ca, cee, lac, mena, seap, sa, ssa",...,"we, seap","we, ssa","we, ssa, ea, sa, na","we, ssa, lac, mena","we, ssa, na","we, ssa, na, sa, ea, mena","we, ssa, sa","we, ssa, sa, mena",we/na,we/na (assumed)
0,[yes/no]_[yes/no]_[yes/no],0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,nan_nan_nan,11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,no (insufficient information)_no (insufficient...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,no evidence due to retraction_no evidence due ...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,no information_no information_no information,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,no_no_no,7184,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6,no_no_not applicable,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,no_no_not assessable,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,no_no_not determinable from given information,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,no_no_not enough information,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
merged_df_unattributable.to_csv(path_equity_precision_llm_folder + '/llm training/epl05_output_Unattributable.csv', index=False)